In [85]:
import re
import os
import pysolr
import tabulate
from datetime import datetime
from pprint import pprint
from IPython.display import HTML, display



In [86]:
# Solr collection url 
SOLR_URL = 'http://solr:8983/solr/cmp269'

QUERIES_FILE_PATH = 'data/Consultas_UTF8.txt'

## Código pra ler os arquivos e separar as queries com base nas tags

In [87]:
def parse_file(fpath, doc_tag):
    '''Reads file line by line and extract the docs'''

    docs = []
    closing_doc_line = "</%s>\n" % doc_tag
    
    with open(fpath) as file:
        doc_as_string = ""

        line = file.readline()
        while (line):
            if (line) == closing_doc_line:
                doc_as_string += line
                
                doc = parse_doc(doc_as_string)
                docs.append(doc)

                doc_as_string = ""
            else:
                doc_as_string += line

            line = file.readline()

    return docs

## Código para fazer o parse da string de cada query e criar um dicionário

In [88]:
def parse_doc(doc_as_string):
    # print(doc_as_string)

    doc_tpl = {
        'num': 'num',
        'PT-title': 'title',
        'PT-desc': 'desc',
        'PT-narr': 'narr',
    }

    doc = {}

    for key in doc_tpl.keys():
        # generates something like:
        # str_pattern = ".*\<DOCID\>(.*)\<\/DOCID\>.*"
        str_pattern = ".*\<%s\>(.*)\<\/%s\>.*" % (key, key)

        # compiles pattern, capture match, updates doc
        re_pattern = re.compile(str_pattern, re.DOTALL)
        match = re_pattern.match(doc_as_string)
        
        if match:
            value = match.group(1).strip()
            solr_key = doc_tpl[key]
            doc.update({solr_key: value})

    # ensures we have all keys..
    for key in doc_tpl.values():
        assert(doc[key] is not None)

    return doc


## Testando várias maneiras diferentes de fazer a query na API do Solr...

In [89]:
def query(params):
    result = sq.search(**params)
    result_lines.append([result.hits, params])


sq = pysolr.Solr(SOLR_URL)


queries = parse_file(QUERIES_FILE_PATH, 'top')

# Vamos testar só com as duas primeiras
queries = queries[:2]

for q in queries:
    
    result_lines = []
    title = q["title"]

    # query inteira e entre aspas, precedida pelo nome do campo ("text_txt_pt:")
    quoted_query = 'text_txt_pt:"%s"' % title

    # cada termo da query precedido pelo nome do campo
    parts = ["text_txt_pt:%s" % p for p in title.split(' ')]
    unquoted_query = '%s' % ' '.join(parts)

    # cada termo da query precedido por "+text_txt_pt:" (repare o sinal +)
    parts = ["+text_txt_pt:%s" % p for p in title.split(' ')]
    plus_query = '%s' % ' '.join(parts)

    
    # default field
    default_field = "text_txt_pt"
    
    # query inteira e entre aspas (sem o nome do campo)
    quoted_query_df = '"%s"' % title

    # query inteira e sem as aspas
    unquoted_query_df = title

    # cada termo da query precedido só pelo sinal de +
    parts = ["+%s" % p for p in title.split(' ')]
    plus_query_df = '%s' % ' '.join(parts)


    # cabeçalho da tabela
    result_lines.append([None, title])
    result_lines.append(['# Results', 'Parameters'])

    ##
    ## Usando o parâmetro query 
    ##
    query({
        "q": quoted_query
    })

    query({
        "q": unquoted_query,
        "q.op": "OR"
    })
    
    query({
        "q": unquoted_query,
        "q.op": "AND"
    })

    query({
        "q": plus_query
    })


    # linha em branco na tabela
    result_lines.append([None,None])

    ##
    ## Usando filter query
    ##
    query({
        "q": "*:*",
        "fq": quoted_query
    })

    query({
        "q": "*:*",
        "q.op": "OR",
        "fq": unquoted_query,
    })

    query({
        "q": "*:*",
        "q.op": "AND",
        "fq": unquoted_query,
    })

    query({
        "q": "*:*",
        "fq": plus_query,
    })


    # linha em branco na tabela
    result_lines.append([None,None])

    ##
    ## Usando default field
    ##

    query({
        "q": quoted_query_df,
        "df": default_field,
    })

    query({
        "q": unquoted_query_df,
        "q.op": "OR",
        "df": default_field,
    })

    query({
        "q": unquoted_query_df,
        "q.op": "AND",
        "df": default_field,
    })
    
    query({
        "q": plus_query_df,
        "df": default_field,
    })

    display(HTML(tabulate.tabulate(result_lines, tablefmt='html')))

print("The end")

,Medicina alternativa
# Results,Parameters
2,"{'q': 'text_txt_pt:""Medicina alternativa""'}"
2616,"{'q': 'text_txt_pt:Medicina text_txt_pt:alternativa', 'q.op': 'OR'}"
41,"{'q': 'text_txt_pt:Medicina text_txt_pt:alternativa', 'q.op': 'AND'}"
41,{'q': '+text_txt_pt:Medicina +text_txt_pt:alternativa'}
,
2,"{'q': '*:*', 'fq': 'text_txt_pt:""Medicina alternativa""'}"
2616,"{'q': '*:*', 'q.op': 'OR', 'fq': 'text_txt_pt:Medicina text_txt_pt:alternativa'}"
41,"{'q': '*:*', 'q.op': 'AND', 'fq': 'text_txt_pt:Medicina text_txt_pt:alternativa'}"
41,"{'q': '*:*', 'fq': '+text_txt_pt:Medicina +text_txt_pt:alternativa'}"


,Sistemas de reforma e pensões na Europa
# Results,Parameters
0,"{'q': 'text_txt_pt:""Sistemas de reforma e pensões na Europa""'}"
11409,"{'q': 'text_txt_pt:Sistemas text_txt_pt:de text_txt_pt:reforma text_txt_pt:e text_txt_pt:pensões text_txt_pt:na text_txt_pt:Europa', 'q.op': 'OR'}"
0,"{'q': 'text_txt_pt:Sistemas text_txt_pt:de text_txt_pt:reforma text_txt_pt:e text_txt_pt:pensões text_txt_pt:na text_txt_pt:Europa', 'q.op': 'AND'}"
0,{'q': '+text_txt_pt:Sistemas +text_txt_pt:de +text_txt_pt:reforma +text_txt_pt:e +text_txt_pt:pensões +text_txt_pt:na +text_txt_pt:Europa'}
,
0,"{'q': '*:*', 'fq': 'text_txt_pt:""Sistemas de reforma e pensões na Europa""'}"
11409,"{'q': '*:*', 'q.op': 'OR', 'fq': 'text_txt_pt:Sistemas text_txt_pt:de text_txt_pt:reforma text_txt_pt:e text_txt_pt:pensões text_txt_pt:na text_txt_pt:Europa'}"
0,"{'q': '*:*', 'q.op': 'AND', 'fq': 'text_txt_pt:Sistemas text_txt_pt:de text_txt_pt:reforma text_txt_pt:e text_txt_pt:pensões text_txt_pt:na text_txt_pt:Europa'}"
0,"{'q': '*:*', 'fq': '+text_txt_pt:Sistemas +text_txt_pt:de +text_txt_pt:reforma +text_txt_pt:e +text_txt_pt:pensões +text_txt_pt:na +text_txt_pt:Europa'}"


The end


## Escolhendo um jeito...

Dá pra ver que há vários jeitos de fazer a mesma query...
Escolhendo uma das opções que traz pelo menos um resultado ambas as queries:

* Sem aspas
* Usando o operador OR
* Indicando o default_field

Ex:

```python
{
    'df': 'text_txt_pt', 
    'q': 'Centrais nucleares', 
    'q.op': 'OR'
}
```




In [90]:
sq = pysolr.Solr(SOLR_URL)

result_lines = []
result_lines.append(['# Results', 'query'])

queries = parse_file(QUERIES_FILE_PATH, 'top')
for q in queries:

    title = q["title"]

    params = {
        "q": title,
        "q.op": "OR",
        "df": "text_txt_pt",
    }
    
    result = sq.search(**params)
    result_lines.append([result.hits, title])
    
    result = None
    
display(HTML(tabulate.tabulate(result_lines, tablefmt='html')))

print("The end")

# Results,query
2616,Medicina alternativa
11409,Sistemas de reforma e pensões na Europa
10121,Países com pena de morte
1406,Danos provocados por terramotos
964,Viciados na Internet
1218,Doença de Creutzfeldt-Jakob
1209,Limpeza étnica nos Balcãs
1940,Legislação anti-tabagista
12,Adivinhação
2,Espectáculos de beneficiência


The end
